# 04.4- Agents



Let's take a look at agents, and expand the capabilities of LLMs.

### AgentType

There are several types of agents, each optimized for what we need. An agent can store past chat information or can enforce each call to the LLM as an independent one.  
It is up to you to decide what you need in your future projects, but here we will focus on agents capable of maintaining a chat.

### Tools

The `tools` are a set of options one uses to give further capabilities to the agent: parsing wikipedia, using google for searches, etc..  
Several models are trained with a 1-2 year delay in data, like chatgpt-3.5. It was only trained with data up to 2022.

In [ ]:
#!pip install numexpr
#!pip install langchainhub
#!pip install langchain_openai

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from IPython.display import Markdown

In [2]:
# Let's create a memory instance for our agent
memory = ConversationBufferMemory(memory_key="chat_history")

In [3]:
llm=ChatOpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm) 
agent_chain = initialize_agent(tools, 
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory)

/home/guimarais/miniconda3/envs/adpro/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/guimarais/miniconda3/envs/adpro/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
answer = agent_chain.invoke(input="What are some good Portuguese food recipes?", handle_parsing_errors=True)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Portuguese cuisine is known for its delicious flavors and diverse dishes. Here are a few popular Portuguese food recipes:

1. Bacalhau à Brás: This is a traditional Portuguese dish made with salted codfish, eggs, onions, and thinly sliced potatoes. It's a flavorful and hearty dish that is often served as a main course.

2. Pastéis de Nata: These are delicious Portuguese custard tarts with a crispy pastry shell and a creamy custard filling. They are a popular dessert in Portugal and can be found in many bakeries.

3. Caldo Verde: This is a traditional Portuguese soup made with potatoes, kale, onions, and garlic. It's a simple and comforting soup that is often served as a starter.

4. Francesinha: This is a hearty sandwich that originated in Porto, Portugal. It typically consists of layers of bread, ham, sausage, and steak, topped with melted cheese and a rich tomato and beer sauce.

5. Arroz de Marisco: Thi

In [5]:
Markdown(answer['output'])

Portuguese cuisine is known for its delicious flavors and diverse dishes. Here are a few popular Portuguese food recipes:

1. Bacalhau à Brás: This is a traditional Portuguese dish made with salted codfish, eggs, onions, and thinly sliced potatoes. It's a flavorful and hearty dish that is often served as a main course.

2. Pastéis de Nata: These are delicious Portuguese custard tarts with a crispy pastry shell and a creamy custard filling. They are a popular dessert in Portugal and can be found in many bakeries.

3. Caldo Verde: This is a traditional Portuguese soup made with potatoes, kale, onions, and garlic. It's a simple and comforting soup that is often served as a starter.

4. Francesinha: This is a hearty sandwich that originated in Porto, Portugal. It typically consists of layers of bread, ham, sausage, and steak, topped with melted cheese and a rich tomato and beer sauce.

5. Arroz de Marisco: This is a flavorful seafood rice dish that is popular in coastal regions of Portugal. It typically includes a variety of seafood such as shrimp, clams, and mussels, cooked with rice and flavored with garlic, onions, and tomatoes.

These are just a few examples of the delicious Portuguese food you can try. I hope you enjoy exploring the flavors of Portugal! Let me know if there's anything else I can help with.

In [6]:
answer = agent_chain.run("Which one of those dishes has the least fat?")

/home/guimarais/miniconda3/envs/adpro/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Among the dishes mentioned, Caldo Verde is likely to have the least fat. Caldo Verde is a traditional Portuguese soup made with potatoes, kale, onions, and garlic. It is typically a simple and comforting soup that does not contain high-fat ingredients like meat or cheese. However, it's worth noting that the fat content can vary depending on the specific recipe and cooking methods used. If you're looking for a low-fat option, Caldo Verde is a good choice.

> Finished chain.


Now the history of our conversation is stored and you can build upon what you have chatted.

In [7]:
agent_chain.run("Give me a grocery shopping list to make that dish")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Sure! Here's a grocery shopping list to make Caldo Verde:

- Potatoes
- Kale
- Onions
- Garlic
- Olive oil
- Salt and pepper

These are the basic ingredients needed to make Caldo Verde. You may also want to consider adding some optional ingredients such as chorizo or linguica sausage for added flavor. Enjoy cooking! Let me know if there's anything else I can assist you with.

> Finished chain.


"Sure! Here's a grocery shopping list to make Caldo Verde:\n\n- Potatoes\n- Kale\n- Onions\n- Garlic\n- Olive oil\n- Salt and pepper\n\nThese are the basic ingredients needed to make Caldo Verde. You may also want to consider adding some optional ingredients such as chorizo or linguica sausage for added flavor. Enjoy cooking! Let me know if there's anything else I can assist you with."

---
### Other applications: Analyzing documents

The Langchain framework has a lot of code already specialized in tasks meant to be done over and over.  
In this example we go over a text file that we have seen before, the play from William Shakespeare.

In [8]:
import requests

# Let's get our Shakespeare back
response = requests.get("https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# There is a size limit for inputs
shakespeare = response.text[0:10000]

In [9]:
from langchain.chains import AnalyzeDocumentChain

In [10]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm)
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [11]:
qa_document_chain.run(
    input_document=shakespeare,
    question="What is this work?",
)

'This work is a play called "Coriolanus" by William Shakespeare.'

In [12]:
qa_document_chain.run(
    input_document=shakespeare,
    question="Describe the main protagonist of this work.",
)

'The main protagonist of this work is Caius Marcius, also known as Coriolanus. He is portrayed as a proud and fierce Roman general who is highly skilled in battle. He is known for his loyalty to Rome and his disdain for the common people, whom he sees as unworthy and beneath him. Marcius is a complex character, driven by his ambition and desire for honor and recognition. However, his arrogance and inability to connect with the common people ultimately lead to his downfall.'

---
### PDF documents

In [13]:
#!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader

In [14]:
#!wget https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf

In [15]:
loader = PyPDFLoader("EN_MTGM14_PrintedRulebook_LR.pdf")
pages = loader.load_and_split()

In [16]:
pages[10]

Document(page_content='Destroy target creature with flying.\n“Let nothing own the skies but the wind. ”\n —Dejara, Giltwood druid\n™ & © 2013 Wizards of the Coast 188/249 Pete VentersInstantOOo1oGPlummet\n129149\nCG16_M14Plummet\n0188_MTGM14S-Spell\n10\nTarget\nWhen you see the word “target” \non a spell or ability, you have to choose one or more things for the spell or ability to affect. You’ll be able to choose only certain kinds of things, such as “target enchantment” or “target creature or player.”\nYou choose the targets for a \nspell when you cast it, and you choose targets for an activated ability when you activate it. If you can’t meet the targeting requirements, you can’t cast the spell or activate the activated ability. Once you choose targets, you can’t change your mind later.\nWhen the spell or ability resolves, it checks the \ntargets to make sure they’re still legal (they’re still there, and they match the requirements stated by the spell or ability). If a target isn’t le

A PDF is usually text+images. Ideally, you send both the images and the the text to the model for analysis.  
But you need a multi-modal model for that, which we will not cover due to time constraints.  
Here, we are just going to get the text and run some simple queries.

In [17]:
pages[10].to_json()['kwargs']['page_content']

'Destroy target creature with flying.\n“Let nothing own the skies but the wind. ”\n —Dejara, Giltwood druid\n™ & © 2013 Wizards of the Coast 188/249 Pete VentersInstantOOo1oGPlummet\n129149\nCG16_M14Plummet\n0188_MTGM14S-Spell\n10\nTarget\nWhen you see the word “target” \non a spell or ability, you have to choose one or more things for the spell or ability to affect. You’ll be able to choose only certain kinds of things, such as “target enchantment” or “target creature or player.”\nYou choose the targets for a \nspell when you cast it, and you choose targets for an activated ability when you activate it. If you can’t meet the targeting requirements, you can’t cast the spell or activate the activated ability. Once you choose targets, you can’t change your mind later.\nWhen the spell or ability resolves, it checks the \ntargets to make sure they’re still legal (they’re still there, and they match the requirements stated by the spell or ability). If a target isn’t legal, the spell or abil

In [18]:
text = []

# Append all pages on a list and then join all list elements
for page in pages:
    text.append(page.to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [19]:
# Number of pages, number of characters
len(text), len(all_text)

(45, 111147)

The document is too big for our model, we need to focus on some pages only.

In [20]:
text = []

# Append all pages on a list and then join all list elements
for pagenumber in range(2,6):
    text.append(pages[pagenumber].to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [21]:
# Number of pages, number of characters
len(text), len(all_text)

(4, 10093)

In [22]:
text

['3\nContents\nSection 1: The Basics\nFive Colors of Mana .......................................... 4\nParts of a Card .............................................. 5\nCard Types ................................................. 5\nGame Zones ................................................ 8\nSection 2: The Building Blocks\nMaking Mana . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\nSpells .................................................... 10\nAbilities .................................................. 12\nAttacking and Blocking  ...................................... 13\nBuilding Your Own Deck ..................................... 15\nThe Golden Rule  ........................................... 15\nSection 3: Playing a Game\nGet a Deck ................................................ 16\nGet a Friend ............................................... 16\nStart the Game ............................................. 16\nParts of the T 

In [23]:
qa_document_chain.run(
    input_document=all_text,
    question="What are the parts of a card?",
)

"The parts of a card include the mana cost, expansion symbol, collector number, power and toughness (for creature cards), type line, card name, text box (which includes abilities and flavor text), and the artist's name."

To analyse more content, one must use models capable of handling more tokens.